In [1]:

import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize

In [2]:
dataset_path = r'C:\Programming Projects\PRODIGY_ML_03\train'

images = []
labels = []

# Define a mapping for labels
label_mapping = {'cats': 0, 'dogs': 1}


In [3]:
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    if not os.path.isdir(category_path):
        continue

    for file_name in os.listdir(category_path):
        image_path = os.path.join(category_path, file_name)
        if not os.path.isfile(image_path):
            continue

        image = cv2.imread(image_path)
        if image is None:  # Check if the image is loaded properly
            continue
            
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = cv2.resize(image, (64, 64))

        features, hog_image = hog(image, orientations=9, pixels_per_cell=(8, 8), 
                                   cells_per_block=(2, 2), visualize=True)
        
        images.append(features)
        labels.append(label_mapping[category])

In [4]:
X = np.array(images)
y = np.array(labels)

In [5]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
}

svm_model = SVC(random_state=42, probability=True)
grid_search = GridSearchCV(svm_model, param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

In [ ]:
print("Best Parameters:", grid_search.best_params_)

y_pred = grid_search.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred, target_names=['cats', 'dogs']))

In [ ]:
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = resize(gray_image, (64, 64))
    features, _ = hog(resized_image, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=True)
    return features.flatten()

In [ ]:
image_path = r'C:\Programming Projects\PRODIGY_ML_03\test\55.jpg'
preprocessed_image = preprocess_image(image_path)
reshaped_image = preprocessed_image.reshape(1, -1)

prediction = grid_search.predict(scaler.transform(reshaped_image))  # Ensure to scale the test image
print("Predicted Label:", prediction)

In [ ]:
img = imread(image_path)
plt.imshow(img)
plt.axis('off')  # Hide axis
plt.show()